# 4. SED Fitting to Derive the Global Properties of Galaxies

In this step, we will perform spectral energy distribution (SED) fitting using [piXedfit](https://github.com/aabdurrouf/piXedfit) to measure the global (i.e., integrated) stellar population properties of high redshift galaxies. In this example, we will focus on galaxies at redshift range of 4.0<z<5.0 from both cropped regions.  

In [1]:
import os, sys
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## 4.1. Select Galaxies and make input photometric data for piXedfit

In this step, we select galaxies at 4.0<z<5.0 from cropped region 1 and 2, then store them into new text files, which will then be inputted to piXedfit for SED fitting and measuring their properties.

In [5]:
# selecting galaxies at 4.0<z<5.0 in cropped region 1
data1 = np.loadtxt('ceers_crop1_photo_pz.cat')
idx1 = np.where((data1[:,17]>4.0) & (data1[:,17]<5.0))
ngals_crop1 = len(idx1[0])
print ("Number of galaxies selected in region 1: %d" % ngals_crop1)

# selecting galaxies at 4.0<z<5.0 in cropped region 2
data2 = np.loadtxt('ceers_crop2_photo_pz.cat')
idx2 = np.where((data2[:,17]>4.0) & (data2[:,17]<5.0))
ngals_crop2 = len(idx2[0])
print ("Number of galaxies selected in region 2: %d" % ngals_crop2)

#==> put into text files: to be inputted to piXedfit
file_out = open('ceers_crop1_photo_pz_4z5.cat','w')
file_out.write("#(1)id (2)x (3)y (4)ra (5)dec (6)f_f115w (7)e_f115w (8)f_f150w (9)e_f150w (10)f_f200w (11)e_f200w (12)f_f277w (13)e_f277w (14)f_f356w (15)e_f356w (16)f_f444w (17)e_f444w (18)photo-z (20)photo-z chi2\n")
file_out.write("# All flux densisties are in units of erg/s/cm^2/Angstrom\n")
for ii in idx1[0]:
    file_out.write("%d " % data1[ii][0])
    for xx in range (1,data1.shape[1]-1):
        file_out.write("%e " % data1[ii][xx])
    file_out.write("%e\n" % data1[ii][data1.shape[1]-1])
file_out.close()
        

#==> put into text files: to be inputted to piXedfit
file_out = open('ceers_crop2_photo_pz_4z5.cat','w')
file_out.write("#(1)id (2)x (3)y (4)ra (5)dec (6)f_f115w (7)e_f115w (8)f_f150w (9)e_f150w (10)f_f200w (11)e_f200w (12)f_f277w (13)e_f277w (14)f_f356w (15)e_f356w (16)f_f444w (17)e_f444w (18)photo-z (20)photo-z chi2\n")
file_out.write("# All flux densisties are in units of erg/s/cm^2/Angstrom\n")
for ii in idx2[0]:
    file_out.write("%d " % data2[ii][0])
    for xx in range (1,data2.shape[1]-1):
        file_out.write("%e " % data2[ii][xx])
    file_out.write("%e\n" % data2[ii][data1.shape[1]-1])
file_out.close()


Number of galaxies selected in region 1: 17
Number of galaxies selected in region 2: 26


## 4.2. Generate model spectra at rest-frame

Script below can be used to generate model spectra at rest-frame. The `save_models_rest_spec` function implements MPI for parallel computation. The script below uses 20 cores (can be changed), therefore it is recommended to execute this script on a cluster.  

In [ ]:
from astropy.cosmology import *

from piXedfit.piXedfit_model import save_models_rest_spec

imf_type = 1                    # Chabrier (2003)
sfh_form = 4                    # double power law SFH form
dust_law = 0                    # Charlot & Fall (2000) dust attenuation law
duste_switch = 0                # turn off dust emission
add_neb_emission = 1            # turn on nebular emission
add_agn = 0                     # turn off AGN dusty torus emission

nmodels = 70000                 # number of models to be generated
nproc = 20                      # number of cores (processors) to be used in calculations 

max_z = 4.0                     # lowest redshift

cosmo1 = FlatLambdaCDM(H0=70.0, Om0=0.3)
age_univ = cosmo1.age(max_z)
max_log_age = np.log10(age_univ.value)

params_range = {'dust1':[0.0,3.0], 'dust2':[0.0,3.0], 'log_age':[-1.0,max_log_age], 
                'log_alpha':[-2.0,2.0], 'log_beta':[-2.0,2.0]}

name_out = 'model_specs_restframe.hdf5'
save_models_rest_spec(imf_type=imf_type, sfh_form=sfh_form, dust_law=dust_law, params_range=params_range,
                        duste_switch=duste_switch, add_neb_emission=add_neb_emission, add_agn=add_agn,
                        nmodels=nmodels, nproc=nproc, name_out=name_out)

## 4.3. Run SED fitting

### SED fitting for galaxies in cropped region 1

In [ ]:
from piXedfit.piXedfit_fitting import singleSEDfit
from piXedfit.piXedfit_fitting import priors

# set of filters
filters = ['jwst_nircam_f115w', 'jwst_nircam_f150w', 'jwst_nircam_f200w', 'jwst_nircam_f277w',
           'jwst_nircam_f356w', 'jwst_nircam_f444w']
nbands = len(filters)

# index of fluxes in the catalog
idx_flux = np.asarray([5, 7, 9, 11, 13, 15])
idx_flux = idx_flux.astype(int)
# index of flux errors in the catalog
idx_flux_err = np.asarray([6, 8, 10, 12, 14, 16])
idx_flux_err = idx_flux_err.astype(int)

# model spectra at rest-frame
models_spec = 'models/model_specs_restframe.hdf5'

# fitting method
fit_method = 'mcmc'

nproc = 20 

# open the photometric data
data = np.loadtxt('ceers_crop1_photo_pz_4z5.cat')
    
for ii in range(0,ngals_crop1):
    # get the SED
    obs_flux = data[ii][idx_flux]
    obs_flux_err = data[ii][idx_flux_err]
    
    # get photometric redshift
    gal_z = data[ii][17]
    
    # define prior for redshift
    ranges = {'z':[gal_z-0.7,gal_z+0.7]}
    pr = priors(ranges)
    params_ranges = pr.params_ranges()
    
    prior1 = pr.gaussian('z',gal_z,0.5)
    params_priors = [prior1]
    
    # name of output FITS file:
    name_out_fits = 'mcmc_crop1_%d.fits' % data[ii][0]
    singleSEDfit(obs_flux, obs_flux_err, filters, models_spec, params_ranges=params_ranges, 
             params_priors=params_priors, fit_method=fit_method, gal_z=None, nrands_z=20, 
             add_igm_absorption=1, igm_type=1, nwalkers=100, nsteps=600, nproc=nproc, initfit_nmodels_mcmc=30000, 
             store_full_samplers=1, name_out_fits=name_out_fits)


### SED fitting for galaxies in cropped region 2

In [ ]:
from piXedfit.piXedfit_fitting import singleSEDfit
from piXedfit.piXedfit_fitting import priors

# set of filters
filters = ['jwst_nircam_f115w', 'jwst_nircam_f150w', 'jwst_nircam_f200w', 'jwst_nircam_f277w',
           'jwst_nircam_f356w', 'jwst_nircam_f444w']
nbands = len(filters)

# index of fluxes in the catalog
idx_flux = np.asarray([5, 7, 9, 11, 13, 15])
idx_flux = idx_flux.astype(int)
# index of flux errors in the catalog
idx_flux_err = np.asarray([6, 8, 10, 12, 14, 16])
idx_flux_err = idx_flux_err.astype(int)

# model spectra at rest-frame
models_spec = 'models/model_specs_restframe.hdf5'

# fitting method
fit_method = 'mcmc'

nproc = 20 

# open the photometric data
data = np.loadtxt('ceers_crop2_photo_pz_4z5.cat')
    
for ii in range(0,ngals_crop2):
    # get the SED
    obs_flux = data[ii][idx_flux]
    obs_flux_err = data[ii][idx_flux_err]
    
    # get photometric redshift
    gal_z = data[ii][17]
    
    # define prior for redshift
    ranges = {'z':[gal_z-0.7,gal_z+0.7]}
    pr = priors(ranges)
    params_ranges = pr.params_ranges()
    
    prior1 = pr.gaussian('z',gal_z,0.5)
    params_priors = [prior1]
    
    # name of output FITS file:
    name_out_fits = 'mcmc_crop2_%d.fits' % data[ii][0]
    singleSEDfit(obs_flux, obs_flux_err, filters, models_spec, params_ranges=params_ranges, 
             params_priors=params_priors, fit_method=fit_method, gal_z=None, nrands_z=20, 
             add_igm_absorption=1, igm_type=1, nwalkers=100, nsteps=600, nproc=nproc, initfit_nmodels_mcmc=30000, 
             store_full_samplers=1, name_out_fits=name_out_fits)


## 4.4. Check fitting results: make diagnostics plots

## 4.5. Make image cutout of the galaxies